In [2]:
from tinygrad import Tensor, nn
from tinygrad.helpers import trange
import pickle
import numpy as np

In [3]:
def unpickle(file):
    with open(file, "rb") as f:
        data = pickle.load(f, encoding="bytes")
    return data

In [4]:
H = W = 32
C = 3
data = unpickle("data/data_batch_1")
X_train = Tensor(data[b"data"].reshape((10000, C, H, W)))
Y_train = Tensor(np.array(data[b"labels"]))

data = unpickle("data/test_batch")
X_test = Tensor(data[b"data"].reshape((10000, C, H, W)))
Y_test = Tensor(np.array(data[b"labels"]))

In [30]:
P = 16
class ViT:
    def __init__(self, embed_dim):
        self.embed_dim = embed_dim
        self.embedding = nn.Conv2d(3, embed_dim, kernel_size=16, stride=16)
        self.cls_token = Tensor.uniform(1, 1, embed_dim)
        self.pos_embedding = Tensor.uniform(1, (H // P) * (W // P) + 1, embed_dim)

    def __call__(self, x):
        ce = Tensor.zeros(x.shape[0], 1, self.embed_dim) + self.cls_token
        x = self.embedding(x).flatten(2).transpose(1, 2)
        x = ce.cat(x, dim=1)
        x += self.pos_embedding
        return x

model = ViT(embed_dim=192)
model(X_train)

<Tensor <UOp METAL (10000, 5, 192) float (<Ops.ADD: 45>, None)> on METAL with grad None>